## Organizing historical capital stock (2000-2020) in long-panel format, converting to current and constant PPP terms, taking care of missing data

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import numpy as np
import pandas as pd
from sliiders import country_level_ypk as ypk_fn
from sliiders import settings as sset
from sliiders.io import save

ALL_ISOS = pd.read_parquet(sset.PATH_ALL_VALID_HIST_CCODES).iloc[:, 0].values
FINAL_ISOS = [i for i in ALL_ISOS if "+" not in i]

## Importing cleaned historical GDP and population (2000-2020) and others

In [3]:
yp_df = pd.read_parquet(sset.PATH_INC_POP_CLEANED)
gdp_pop_df = pd.read_parquet(sset.PATH_INC_POP_AGG)
pwt100 = pd.read_excel(sset.PATH_PWT_RAW).rename(columns={"countrycode": "ccode"})
pwt100.set_index(["ccode", "year"], inplace=True)

## Using `rnna` (constant 2017 PPP USD) and `cn` (current PPP, 2017 USD) from PWT 10.0

We import available `rnna` and `cn` information and use perpetual inventory method for 2020.

In [4]:
# basic cleaning for France (representing France + overseas departments)
pwt100 = pd.read_excel(sset.PATH_PWT_RAW).rename(columns={"countrycode": "ccode"})
pwt100.loc[pwt100.ccode == "FRA", "ccode"] = sset.FRA_OVERSEAS_DEPT
pwt100.set_index(["ccode", "year"], inplace=True)

# extracting relevant information; PWT doesn't have 2020
k_df = pwt100.loc[
    (slice(None), list(range(2000, 2020))),
    ["rnna", "cn", "csh_i", "delta", "rgdpna", "cgdpo"],
]
pwt_all_empty, pwt_partial_empty, k_df_2020 = [], [], []
for i in k_df.reset_index()["ccode"].unique():
    if pd.isnull(k_df.loc[i, "rnna"]).all():
        pwt_all_empty.append(i)
        continue
    else:
        # checking if any is partially missing 2000-2020, but none was missing
        if pd.isnull(k_df.loc[i, "rnna"]).any() or pd.isnull(k_df.loc[i, "cn"]).any():
            pwt_partial_empty.append(i)
        # pertual inventory for 2020 values
        i_2020 = k_df.loc[(i, 2019), "rgdpna"] * k_df.loc[(i, 2019), "csh_i"]
        ci_2020 = k_df.loc[(i, 2019), "cgdpo"] * k_df.loc[(i, 2019), "csh_i"]
        k_2020 = (
            k_df.loc[(i, 2019), "rnna"] * (1 - k_df.loc[(i, 2019), "delta"]) + i_2020
        )
        ck_2020 = (
            k_df.loc[(i, 2019), "cn"] * (1 - k_df.loc[(i, 2019), "delta"]) + ci_2020
        )
    k_df_2020.append([i, 2020, k_2020, ck_2020])

# excluding all-empty cases and attaching 2020 values
k_df = pd.concat(
    [
        k_df.loc[
            ~k_df.index.get_level_values("ccode").isin(pwt_all_empty), ["rnna", "cn"]
        ],
        pd.DataFrame(k_df_2020, columns=["ccode", "year", "rnna", "cn"]).set_index(
            ["ccode", "year"]
        ),
    ]
).sort_index()

# turn into ones
k_df = k_df * 1e6

# set source variable
k_df["capital_source"] = "PWT_PIM"

print("The following countries are in PWT but with no rnna info:", pwt_all_empty)

The following countries are in PWT but with no rnna info: ['CUW', 'GUY', 'SXM']


In [5]:
# creating rnna_19 and cn_19
usd_17_19 = pwt100.loc[("USA", 2019), "pl_n"] / pwt100.loc[("USA", 2017), "pl_n"]
for i in ["rnna", "cn"]:
    if i == "cn":
        k_df[i + "_19"] = k_df[i] * usd_17_19
    k_df.rename(columns={i: i + "_17"}, inplace=True)

k_df = k_df.join(
    k_df.loc[(slice(None), 2019), "rnna_17"]
    .rename("r_2019")
    .reset_index()
    .drop(["year"], axis=1)
    .set_index(["ccode"])
)
k_df["r_gr"] = k_df["rnna_17"].div(k_df["r_2019"])
k_df = k_df.join(
    k_df.loc[(slice(None), 2019), "cn_19"]
    .rename("c_2019")
    .reset_index()
    .drop(["year"], axis=1)
    .set_index(["ccode"])
)
k_df["rnna_19"] = k_df[["r_gr", "c_2019"]].prod(axis=1)

k_df.drop(["r_gr", "c_2019", "r_2019"], inplace=True, axis=1)

## NFW-to-GDP ratios, 2000-2020

**NFW (non-financial wealth)** values (current, non-PPP) are from Credit Suisse GWDB, and **current, non-PPP GDP** values are from UN SNA AMA (which GWDB cites as "UNSTAT").

In [6]:
# importing GWDB, creating NFW values in millions of USD
gwdb = pd.read_parquet(sset.PATH_GWDB_INT)

# drop the aggregate values
gwdb = gwdb.drop(
    np.concatenate((gwdb.gwdb_region.unique(), ["World"])),
    level="ccode",
    errors="ignore",
)

# calculate nfw
gwdb["nfw"] = gwdb[["nonfinancial_wealth_per_adult", "n_adults"]].prod(axis=1)

# get UNSTATS subregion mapping
region_mapping = (
    pd.read_parquet(sset.PATH_UN_REGION_DATA_INT)
    .reset_index()
    .set_index("ccode")
    .subregion
)

# some manual additions to match other countries
region_mapping = pd.concat(
    (
        region_mapping,
        pd.Series(
            {
                "ZNC": region_mapping.CYP,
                sset.FRA_OVERSEAS_DEPT: region_mapping.FRA,
                "TWN": region_mapping.CHN,
            },
            name="subregion",
        ).rename_axis("ccode"),
    )
)

gdp_data = gdp_pop_df.filter(like="gdp_nom")
gdppc_data = (
    gdp_pop_df.filter(like="gdppc_nom")
    .rename(columns=lambda x: x.replace("gdppc_nom", "gdp_nom"))
    .drop(columns=gdp_data.columns, errors="ignore")
)
gdp_da = pd.concat((gdppc_data.mul(yp_df["pop"], axis=0), gdp_data), axis=1).to_xarray()

for src in ["imf", "wb", "aus_parl", "pcn_gov", "bes_gov", "st_helena_gov", "cerom"]:
    gdp_da["un_gdp_nom"] = ypk_fn.smooth_fill(
        gdp_da["un_gdp_nom"],
        gdp_da[f"{src}_gdp_nom"],
        time_dim="year",
        other_dim="ccode",
    )
gdp_df = (
    gdp_da.un_gdp_nom.rename("gdp_nom_current")
    .sel(year=sset.HISTORICAL_YEARS)
    .to_series()
    .dropna()
)

# use our final list of countries
gdp_df = gdp_df.reindex(yp_df.index).drop(sset.UNINHABITED_ISOS, level="ccode")

gdp_df = gdp_df.to_frame().join(region_mapping, on="ccode", how="left")
assert gdp_df.subregion.notnull().all()

## Apply NFW-to-GDP ratios and use these to fill PWT-PIM results

In [7]:
gwdb_yrs = gwdb.index.get_level_values("year")
gwdb_min, gwdb_max = gwdb_yrs.min(), gwdb_yrs.max()
merged = (
    gdp_df.loc[(slice(None), slice(gwdb_min, gwdb_max)), :]
    .reset_index()
    .set_index(["ccode", "year"])
    .join(gwdb, how="outer")
)

ratios = (merged.nfw / merged.gdp_nom_current).rename("ratio")

# handle french departments so that they specifically have France's ratio
ratios.loc[sset.FRA_OVERSEAS_DEPT.split("+")[1:]] = (
    ratios.loc[sset.FRA_OVERSEAS_DEPT.split("+")[1:]]
    .to_xarray()
    .fillna(ratios.loc["FRA"].to_xarray())
    .to_series()
    .reindex(ratios.loc[sset.FRA_OVERSEAS_DEPT.split("+")[1:]].index)
)

# interpolate
rat_xr = ratios.to_xarray()
ratios = rat_xr.interpolate_na("year", fill_value="extrapolate").to_series()

# fill missing with regional values
regional_ratios = (
    merged.dropna(how="any", subset=["nfw", "gdp_nom_current"])
    .groupby(["subregion", "year"])[["nfw", "gdp_nom_current"]]
    .sum()
)
regional_ratios = (
    (regional_ratios.nfw / regional_ratios.gdp_nom_current)
    .rename("regional_ratio")
    .to_xarray()
    .reindex(year=rat_xr.year)
    .interpolate_na("year", fill_value="extrapolate")
    .to_series()
)

# handle fact that micronesia is missing (assume ratios same as melanesia)
regional_ratios = pd.concat(
    (
        regional_ratios,
        regional_ratios.loc["Melanesia"]
        .reset_index()
        .assign(subregion="Micronesia")
        .set_index(["subregion", "year"])
        .regional_ratio,
    )
)

merged_ratios = (
    ratios.to_frame()
    .join(merged.subregion.groupby("ccode").first())
    .dropna(subset="subregion")
    .join(regional_ratios, on=["subregion", "year"], how="left")
)
merged_ratios["ratio"] = merged_ratios.ratio.fillna(merged_ratios.regional_ratio)
merged_ratios = merged_ratios.drop(columns="regional_ratio")

# add capital source info
merged_ratios["capital_source"] = "GWDB_nfw_gdp_ratio:regional"
merged_ratios.loc[
    ratios.index[ratios.notnull()], "capital_source"
] = "GWDB_nfw_gdp_ratio"
merged_ratios.loc[
    sset.FRA_OVERSEAS_DEPT.split("+")[1:], "capital_source"
] = "GWDB_nfw_gdp_ratio:France"

# check that all are filled
assert merged_ratios.ratio.notnull().all()

# get the NFW:GDP estimated values
gdp_to_nfw = (
    yp_df[["rgdpna_17", "rgdpna_19", "cgdpo_17", "cgdpo_19"]]
    .mul(merged_ratios.ratio, axis=0)
    .rename(columns=lambda x: x.replace("gdpna", "nna").replace("gdpo", "n"))
    .to_xarray()
    .sel(year=sset.HISTORICAL_YEARS)
)

# smooth fill the PWT values with the NFW:GDP values
k_da = k_df.reindex(merged_ratios.index).to_xarray().sel(year=sset.HISTORICAL_YEARS)
for v in gdp_to_nfw.data_vars:
    k_da[v] = ypk_fn.smooth_fill(
        k_da[v],
        gdp_to_nfw[v],
        time_dim="year",
        other_dim="ccode",
    )
assert k_da.drop_vars("capital_source").to_array().notnull().all()

# turn into dataframe
k_df_clean = k_da.to_dataframe().fillna(
    {"capital_source": merged_ratios.capital_source}
)

/tmp/ipykernel_14064/231488093.py:91: DeprecationWarning: dropping variables using `drop` is deprecated; use drop_vars.
  assert k_da.drop("capital_source").to_array().notnull().all()


## Special cases from sources other than PWT and GWDB

### Cuba (`CUB`), using Berlemann and Wesselhoft (2017) and WB WDI I/Y ratio

In [10]:
# from Figure 2 of Berlemann and Wesselhoft (2017); 2014 dep. rate approx. 5 percent
berl_dep = 0.05

# WB WDI I/Y ratio
cub_wb = (
    pd.read_parquet(
        sset.DIR_WB_WDI_RAW / "wdi_pop_iy_gdp.parquet",
        filters=[("ccode", "=", "CUB")],
        columns=["NE.GDI.FTOT.ZS"],
    )
    .droplevel("country")["NE.GDI.FTOT.ZS"]
    .rename("iy_ratio")
)

# take the ratio of Cuba to USA
berlemann = (
    pd.read_excel(sset.DIR_YPK_RAW / "Berlemann_Wesselhoft_2017.xlsx")
    .rename(columns={"Country Name": "country"})
    .set_index(["country"])
)
cub_usa = berlemann.loc[["Cuba", "United States"], list(range(1960, 2017))]
cub_usa.loc["Cuba", list(range(1960, 2017))] /= cub_usa.loc[
    "United States", list(range(1960, 2017))
].values

# subsetting for those that have Cuba capital stock (Cuba-USA ratios being over 0)
cub_years = [x for x in cub_usa.columns if cub_usa.loc["Cuba", x] > 0]
cub_usa_ratio = pd.DataFrame(
    data={"year": cub_years, "ratio": cub_usa.loc["Cuba", cub_years].values}
).set_index(["year"])

# apply this to USA capital
cub = k_df_clean.loc[["USA"], ["rnna_17"]].join(cub_usa_ratio).reset_index()
cub_msg = "Berlemann:CUB_USA_ratio + PWT:USA"
cub["capital_source"] = cub_msg
cub["ccode"] = "CUB"
cub.set_index(["ccode", "year"], inplace=True)

# apply PIM
cub["rnna_17"] = cub[["rnna_17", "ratio"]].prod(axis=1)
for yr in np.sort(np.setdiff1d(sset.HISTORICAL_YEARS, cub_years)):
    pyr = yr - 1
    yr_inv = cub_wb.loc[pyr] / 100 * yp_df.loc[("CUB", pyr), "rgdpna_17"]
    cub.loc[("CUB", yr), "rnna_17"] = (
        cub.loc[("CUB", pyr), "rnna_17"] * (1 - berl_dep) + yr_inv
    )
    cub.loc[("CUB", yr), "capital_source"] = cub_msg + "+ PIM:WB_iyratio"

# apply neutral assumption, so rnna = cn
cub["cn_17"] = cub["rnna_17"]
cub["cn_19"] = cub["cn_17"] * usd_17_19
cub["rnna_19"] = cub["cn_19"]
cub = cub.drop(["ratio"], axis=1)

### North Korea (`PRK`) information from the Bank of Korea report (capital-to-GDP ratio)

`PRK` capital-to-GDP ratio estimates are provided in [Pyo, Cho, and Kim (2020, p. 26, Figure 5)](https://papers.ssrn.com/sol3/papers.cfm?abstract_id=3727104) for the years 1960-2018 (see Figure 2). We combine these with the investment-to-GDP ratio of 2018 (in the same paper, approx. 0.22; we use this for 2019 as well), depreciation rate of 0.033 and 0.1347 for construction capital and facility capital (p. 13), construction share of capital as 358/391 (p. 28; rest being facility capital), and the GDP values that we cleaned (in `yp_df`) to apply PIM and get capital stock of 2019-2020.

In [11]:
# 2000-2018
prk_capital_to_gdp = np.array(
    [
        2.89,
        2.83,
        2.86,
        2.89,
        2.94,
        2.89,
        3.08,
        3.17,
        3.14,
        3.24,
        3.33,
        3.36,
        3.39,
        3.42,
        3.44,
        3.53,
        3.5,
        3.69,
        3.91,
    ]
)
prk_capital_to_gdp = pd.DataFrame(
    data={"year": range(2000, 2019), "ky": prk_capital_to_gdp}
).set_index(["year"])
prk_invest_to_gdp_2018 = 0.22
prk_delta = [0.033, 0.1347]

# construction to (facility + construction approx.= total capital)
prk_construct_ratio = 358 / (358 + 33)

# attaching k-to-y ratio, applying the ratio to GDP, and PIM for 2019-20
prk_y = yp_df.loc[["PRK"], ["rgdpna_17"]].join(prk_capital_to_gdp)
prk_y["rnna_17"] = prk_y[["rgdpna_17", "ky"]].prod(axis=1)
for yr in [2019, 2020]:
    invest = prk_invest_to_gdp_2018 * prk_y.loc[("PRK", yr - 1), "rgdpna_17"]
    prev = prk_y.loc[("PRK", yr - 1), "rnna_17"]
    after_d = prev * prk_construct_ratio * (1 - prk_delta[0]) + prev * (
        1 - prk_construct_ratio
    ) * (1 - prk_delta[1])
    prk_y.loc[("PRK", yr), "rnna_17"] = after_d + invest

# cgdpo_17, cgdpo_19, and rgdpna_19; use neutral assumption
prk_y["cn_17"] = prk_y["rnna_17"]
prk_y["cn_19"] = prk_y["cn_17"] * usd_17_19
prk_y["rnna_19"] = prk_y["cn_19"]
prk_y["capital_source"] = "BOK_kyratio + MPD_GDP"
prk_y.loc[("PRK", [2019, 2020]), "capital_source"] = "BOK_kyratio_PIM + MPD_GDP"

prk_y = prk_y.drop(columns=["ky", "rgdpna_17"])

### Gather all capital

In [12]:
k_df_clean = pd.concat(
    (k_df_clean.drop(["CUB", "PRK"], level="ccode"), prk_y, cub)
).sort_index()

In [13]:
# add uninhabited countries
uninhabited = pd.DataFrame(
    0,
    index=pd.MultiIndex.from_product(
        (sset.UNINHABITED_ISOS, sset.HISTORICAL_YEARS), names=["ccode", "year"]
    ),
    columns=k_df_clean.columns,
)
uninhabited["capital_source"] = "uninhabited"
k_df_clean = pd.concat((k_df_clean, uninhabited))

In [14]:
assert k_df_clean.index.is_unique
assert k_df_clean.notnull().all().all()

## Investment-to-GDP (I/Y) ratios and depreciation rates (delta)

### Delta from PWT 10.0

In [15]:
pwt_d = (
    pwt100.sort_index()
    .loc[pwt100.index.get_level_values("year").isin(sset.HISTORICAL_YEARS), ["delta"]]
    .copy()
    .dropna()
)
pwt_d["delta_source"] = "PWT"

# copy 2019 deltas for 2020
pwt_d_2020 = pwt_d.loc[(slice(None), 2019), :].reset_index()
pwt_d_2020["year"] = 2020
pwt_d_2020.set_index(["ccode", "year"], inplace=True)
pwt_d_2020["delta_source"] = "PWT_copy2019"

# merging
pwt_d = pd.concat([pwt_d, pwt_d_2020])

### Average world delta, 2000-2020 (weighted average using capital stock values)

In [16]:
# attach the capital values to delta
pwt_d = pwt_d.join(k_df_clean.rnna_17)
pwt_d["delta_x_k"] = pwt_d["delta"].mul(pwt_d["rnna_17"])

# weighted average of deltas
world_d = pwt_d.reset_index().groupby(["year"]).sum()[["rnna_17", "delta_x_k"]]
world_d["delta"] = world_d["delta_x_k"].div(world_d["rnna_17"])

### Using average world delta for countries missing those values

Note that delta calculated in `pwt_d` for `FRA` is technically that of `FRA` and the 5 overseas departments combined.

In [17]:
# countries that aren't in `pwt_d`
msng_delta = np.setdiff1d(
    yp_df.reset_index().ccode.unique(), pwt_d.reset_index().ccode.unique()
)
world_avg_d = []
for m in msng_delta:
    m_df = world_d[["delta"]].reset_index()
    m_df["ccode"] = m
    world_avg_d.append(m_df.set_index(["ccode", "year"]))
world_avg_d = pd.concat(world_avg_d, axis=0)

# gathering all
delta = pd.concat([pwt_d[["delta", "delta_source"]], world_avg_d], axis=0).sort_index()
delta.loc[pd.isnull(delta.delta_source), "delta_source"] = "PWT_world_avg"

### I/Y ratio from PWT 10.0 and WB WDI (former as the primary)

We will also copy 2019 values to be 2020 values for completeness when 2020 values are missing. Again, `iy` calculated in `iy_df` for `FRA` is technically that of `FRA` and the 5 overseas departments combined.

In [19]:
iy_df

iy iy_source    wdi_iy
year ccode country                                                  
2000 ABW   Africa Eastern and Southern  0.299764       PWT  0.176920
           Albania                      0.299764       PWT  0.319066
           Algeria                      0.299764       PWT  0.206772
           Arab World                   0.299764       PWT  0.191315
           Argentina                    0.299764       PWT  0.161926
...                                          ...       ...       ...
2019 ZWE   Viet Nam                     0.077089       PWT  0.303626
           West Bank and Gaza           0.077089       PWT  0.252371
           World                        0.077089       PWT  0.259891
           Zambia                       0.077089       PWT  0.357566
           Zimbabwe                     0.077089       PWT  0.106994

[753617 rows x 3 columns]

In [20]:
# using PWT as main, WB WDI as secondary
pwt_iy = pwt100.sort_index()["csh_i"].rename("iy").dropna().to_frame()
pwt_iy["iy_source"] = "PWT"
iy_df = pwt_iy.join(
    pd.read_parquet(sset.DIR_WB_WDI_RAW / "wdi_pop_iy_gdp.parquet")
    .reset_index()
    .set_index(["ccode", "year"])["NE.GDI.FTOT.ZS"]
    .rename("wdi_iy")
    .dropna(),
    how="outer",
)
iy_df = iy_df.loc[
    iy_df.index.get_level_values("ccode").isin(ALL_ISOS)
    & iy_df.index.get_level_values("year").isin(sset.HISTORICAL_YEARS)
].sort_index()
iy_df["wdi_iy"] /= 100
iy_df.loc[pd.isnull(iy_df.iy) & ~pd.isnull(iy_df.wdi_iy), "iy_source"] = "WB"
iy_df.loc[pd.isnull(iy_df.iy), "iy"] = iy_df.loc[pd.isnull(iy_df.iy), "wdi_iy"].values

# copy 2019 to 2020 if missing
iy_2020 = []
for iso in iy_df.reset_index().ccode.unique():
    iso_yrs = iy_df.loc[iso, :].reset_index().year.unique()
    if (2020 not in iso_yrs) and (2019 in iso_yrs):
        iy_19 = iy_df.loc[(iso, 2019), "iy"]
        source_19 = iy_df.loc[(iso, 2019), "iy_source"]
        iy_2020.append([iso, 2020, iy_19, source_19 + "_copy"])
iy_df = pd.concat(
    [
        iy_df,
        pd.DataFrame(iy_2020, columns=["ccode", "year", "iy", "iy_source"]).set_index(
            ["ccode", "year"]
        ),
    ]
)[["iy", "iy_source"]].sort_index()

### Backward-inducing I/Y ratio using capital stock (`rnna_17`), GDP (`rgdpna_17`), and delta values for missing country-years
Use this formula:
$$ \frac{I_{t-1}}{Y_{t-1}} = \frac{K_{t} - (1 - \delta_{t-1})K_{t-1}}{Y_{t-1}} $$

In [21]:
# dividing those that need induction vs. not
full_iso, partial_iso = [], []
for i in iy_df.index.get_level_values("ccode").unique():
    i_yrs = iy_df.loc[i, :].dropna().reset_index().year.unique()
    if len(np.setdiff1d(sset.HISTORICAL_YEARS, i_yrs)) > 0:
        partial_iso.append(i)
    else:
        full_iso.append(i)

iy_df_full = iy_df.loc[(full_iso, sset.HISTORICAL_YEARS), :].copy()
iy_df_partial = iy_df.loc[partial_iso, :].copy()

#### Backward induction for partial cases

In [22]:
def _induction_iy_ratio(
    iso, iy_info=iy_df_partial, y_info=yp_df, k_info=k_df_clean, delta_info=delta
):
    msng_yrs = sset.HISTORICAL_YEARS[::-1]
    if iy_info is not None:
        msng_yrs = np.sort(
            np.setdiff1d(
                sset.HISTORICAL_YEARS, iy_info.loc[iso].reset_index().year.unique()
            )
        )[::-1]

    iy_msng = []
    for i in msng_yrs:
        if i == 2020:
            continue
        y_t1 = y_info.loc[(iso, i), "rgdpna_17"]
        d_t1 = delta_info.loc[(iso, i), "delta"]
        k_t = k_info.loc[(iso, i + 1), "rnna_17"]
        k_t1 = k_info.loc[(iso, i), "rnna_17"]
        iy_t1 = (k_t - (1 - d_t1) * k_t1) / y_t1
        iy_msng.append([iso, i, iy_t1])
    iy_msng = pd.DataFrame(iy_msng, columns=["ccode", "year", "iy"])
    iy_msng["iy_source"] = "backward_PIM"

    if 2020 in msng_yrs:
        iy_2020 = iy_msng.loc[iy_msng.year == 2019, :].copy()
        iy_2020["year"] = 2020
        iy_2020["iy_source"] = "backward_PIM_copy"
        iy_msng = pd.concat([iy_msng, iy_2020])
    iy_msng.set_index(["ccode", "year"], inplace=True)

    if iy_info is not None:
        return pd.concat([iy_info.loc[[iso], :], iy_msng]).sort_index()
    return iy_msng.sort_index()

In [23]:
iy_df_partial_clean = []
for i in partial_iso:
    iy_df_partial_clean.append(_induction_iy_ratio(i))
iy_df_partial_clean = pd.concat(iy_df_partial_clean, axis=0).sort_index()

#### Backward induction for countries with no I/Y ratio

In [24]:
msng_iso = np.setdiff1d(FINAL_ISOS, np.union1d(full_iso, partial_iso))
iy_df_msng = []
for i in msng_iso:
    if i not in sset.UNINHABITED_ISOS:
        iy_df_msng.append(_induction_iy_ratio(i, None))
    else:
        uninhabited = iy_df_full.loc[["USA"], :].reset_index()
        uninhabited["ccode"] = i
        uninhabited["iy"] = 0
        uninhabited["iy_source"] = "uninhabited"
        iy_df_msng.append(uninhabited.set_index(["ccode", "year"]))
iy_df_msng = pd.concat(iy_df_msng, axis=0).sort_index()

# merging with others
iy_df_all = pd.concat([iy_df_partial_clean, iy_df_full, iy_df_msng]).sort_index()
iyd_df = iy_df_all.join(delta, how="left")

## Capital ratio by category

In the case of missing data, we will again use yearly global average and mark those as so.

In [26]:
# captial information
pwt_capital = (
    pd.read_excel(sset.PATH_PWT_RAW.parent / "pwt_K_detail.xlsx")
    .rename(columns={"countrycode": "ccode"})
    .dropna()
)
capital_vals = ["Nc_Struc", "Nc_Mach", "Nc_TraEq", "Nc_Other"]
pwt_capital = pwt_capital.set_index(["ccode", "year"])[capital_vals]
for i in capital_vals:
    pwt_capital[i] = pwt_capital[i].astype("float64")

# ratio of capital in each category to total capital
pwt_capital["total_cap"] = pwt_capital[capital_vals].sum(axis=1)
pwt_capital["movable_cap"] = pwt_capital[capital_vals[1:]].sum(axis=1)
rationames = []
for i in capital_vals:
    name = i.replace("Nc_", "k_").lower() + "_ratio"
    rationames.append(name)
    pwt_capital[name] = pwt_capital[i].div(pwt_capital["total_cap"])
rationames_nos = [x for x in rationames if "struc" not in x]
pwt_capital["k_movable_ratio"] = pwt_capital[rationames_nos].sum(axis=1)
pwt_capital = pwt_capital.loc[
    pwt_capital.index.get_level_values("year").isin(sset.HISTORICAL_YEARS), :
].copy()
pwt_capital["k_ratio_source"] = "PWT"

# attaching 2020 values
pwt_capital_2020 = pwt_capital.loc[
    pwt_capital.index.get_level_values("year") == 2019, :
].reset_index()
pwt_capital_2020["k_ratio_source"] = "PWT_copy2019"
pwt_capital_2020["year"] = 2020
pwt_capital_2020.set_index(["ccode", "year"], inplace=True)
pwt_capital = pd.concat([pwt_capital, pwt_capital_2020], axis=0)

# global average, 2000-2020
global_mov_ratio = (
    pwt_capital.reset_index().groupby("year").sum()[["movable_cap", "total_cap"]]
)
global_mov_ratio["k_movable_ratio"] = global_mov_ratio["movable_cap"].div(
    global_mov_ratio["total_cap"]
)

In [27]:
# attaching the movable ratios for countries not in `pwt_capital`, except uninhabited
msng_k_mov_ratio = []
for i in np.setdiff1d(FINAL_ISOS, pwt_capital.reset_index()["ccode"].unique()):
    if i in sset.UNINHABITED_ISOS:
        i_df = pwt_capital.loc[["USA"], ["k_movable_ratio"]].reset_index()
        i_df["ccode"], i_df["k_ratio_source"] = i, "uninhabited"
        i_df["k_movable_ratio"] = 0.0
    else:
        i_df = global_mov_ratio.reset_index()
        i_df["ccode"], i_df["k_ratio_source"] = i, "PWT_world_avg"
    msng_k_mov_ratio.append(i_df.set_index(["ccode", "year"]))

# merging
k_mov_ratio = pd.concat([pwt_capital] + msng_k_mov_ratio)[
    ["k_movable_ratio", "k_ratio_source"]
]

## Merging all, and creating capital (`rnna_19`, `cn_19`) and population (`pop`) scales

Scales are made in reference to 2019 values.

In [28]:
# merge all
ypk_df = k_mov_ratio.copy()
for i in [iyd_df, k_df_clean, yp_df]:
    ypk_df = ypk_df.join(i, how="left")
assert ypk_df.dropna().shape[0] == len(sset.HISTORICAL_YEARS) * len(FINAL_ISOS)

for i in ["rnna_19", "cn_19", "pop"]:
    i_df = (
        ypk_df.loc[ypk_df.index.get_level_values("year") == 2019, i]
        .rename("scale_base")
        .reset_index()
    )
    i_df.drop(["year"], inplace=True, axis=1)
    i_df.set_index(["ccode"], inplace=True)
    ypk_df = ypk_df.join(i_df)
    ypk_df[f"{i}_scale"] = ypk_df[i].div(ypk_df["scale_base"])
    ypk_df.drop(["scale_base"], axis=1, inplace=True)

## Reordering and exporting

In [29]:
histinfo_columns = [
    "pop_source",
    "gdppc_source",
    "c_to_r_source",
    "capital_source",
    "iy_source",
    "k_ratio_source",
    "delta_source",
    "pop",
    "pop_scale",
    "rgdpna_pc_17",
    "rgdpna_17",
    "rgdpna_pc_19",
    "rgdpna_19",
    "cgdpo_pc_17",
    "cgdpo_17",
    "cgdpo_pc_19",
    "cgdpo_19",
    "iy",
    "k_movable_ratio",
    "delta",
    "rnna_17",
    "rnna_19",
    "rnna_19_scale",
    "cn_17",
    "cn_19",
    "cn_19_scale",
]

ypk_df.rename(columns={"gdp_unit": "gdp_capital_unit"}, inplace=True)
# cleaning up for uninhabited cases
for i in ["rnna_19_scale", "cn_19_scale", "pop_scale"]:
    ypk_df.loc[pd.isnull(ypk_df[i]), i] = 0

ypk_df = ypk_df[histinfo_columns]

In [33]:
save(ypk_df, sset.PATH_EXPOSURE_YPK_COUNTRY_HIST_INT)